## Introduction
In this section of the workflow, we will obtain the metabolomics data and apply filtering options, to create a dataset ready for further statistical and pathway analysis.

## First, we setup the required libraries to get started.


In [1]:
# check if libraries are already installed > otherwise install it
if(!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager",repos = "http://cran.us.r-project.org")
if(!"dplyr" %in% installed.packages()) BiocManager::install("dplyr")
if(!"stringr" %in% installed.packages()){install.packages("stringr")}
#load libraries
library(dplyr)
library(stringr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Second, we download the required data, read the metadata and filter out not-relevant data.


In [9]:
#Library to download data from online files:
if(!"downloader" %in% installed.packages()){install.packages("downloader")}
require(downloader)

##Download metadata, extract metabolomics sample IDs, location and disorders.
if(file.exists("data/hmp2_metadata.csv")){print("Metadata already downloaded")}else{
fileUrl <- "https://ibdmdb.org/tunnel/products/HMP2/Metadata/hmp2_metadata.csv?accessType=DOWNLOAD"
require(downloader)
download(fileUrl, "data/hmp2_metadata.csv", mode = "wb")
}

#read metadata file
metaData <- read.csv("data/hmp2_metadata.csv",stringsAsFactors = FALSE)
#filter out by data type and week number
metaDataMBX <- subset(metaData, metaData$data_type == "metabolomics" )
#we need to have the samples which has same visit number
metaDataMBX<- subset(metaDataMBX, metaDataMBX$visit_num == 4)
#we should match transcriptomics (htx) samples and metabolomics (mbx) samples with participantID
#but samples are given by their externalID in mbx file so we should keep them both
#select columns which will be used
metaDataMBX <- metaDataMBX %>% dplyr::select(External.ID,Participant.ID,diagnosis)
#rename columns of metaDataMBX
colnames(metaDataMBX) <- c("ExternalID","ParticipantID","disease" )

#download and read metabolomics peak intensity data
if(file.exists("data/metabolomics.csv.gz")){print("Metabolomics zipped data already downloaded")}else{
fileUrl <- "https://ibdmdb.org/tunnel/products/HMP2/Metabolites/1723/HMP2_metabolomics.csv.gz?accessType=DOWNLOAD"
download(fileUrl, "data/metabolomics.csv.gz", mode = "wb")
}

#Note: if the URL download does not work, the zipped file is located on GitHub to continue the rest of this script.
if(file.exists("data/metabolomics.csv")){print("Unzipped Metabolomics data already downloaded")}else{
if(!"R.utils" %in% installed.packages()){install.packages("R.utils")}
library(R.utils)
gunzip("data/metabolomics.csv.gz", remove=FALSE)
}
head(metaDataMBX)
metaDataMBX$disease
mbxData <- read.csv("data/metabolomics.csv")
#delete not used columns
mbxData = subset(mbxData, select = -c(1,2,3,4,7) )


[1] "Metadata already downloaded"
[1] "Metabolomics zipped data already downloaded"
[1] "Unzipped Metabolomics data already downloaded"


,ExternalID,ParticipantID,disease
523,CSM5FZ3N,C3001,CD
529,CSM5FZ3T,C3002,CD
535,CSM5FZ48,C3003,UC
540,CSM5FZ4A,C3004,UC
545,CSM5FZ4O,C3005,UC
550,CSM5MCTZ,C3006,UC


[1] "CD"     "CD"     "UC"     "UC"     "UC"     "UC"     "CD"     "CD"    
 [9] "CD"     "UC"     "CD"     "UC"     "UC"     "CD"     "CD"     "nonIBD"
[17] "CD"     "CD"     "UC"     "CD"     "UC"     "UC"     "CD"     "UC"    
[25] "CD"     "UC"     "CD"     "CD"     "CD"     "CD"     "CD"     "nonIBD"
[33] "nonIBD" "UC"     "nonIBD" "CD"     "nonIBD" "CD"     "nonIBD" "UC"    
[41] "nonIBD" "nonIBD" "CD"     "UC"     "UC"     "CD"     "UC"     "nonIBD"
[49] "CD"     "CD"     "CD"     "CD"     "CD"     "nonIBD" "nonIBD" "nonIBD"
[57] "nonIBD" "nonIBD" "nonIBD" "UC"     "nonIBD" "nonIBD" "nonIBD" "nonIBD"
[65] "CD"     "CD"     "CD"     "UC"     "CD"     "UC"

## Third, we perform data extraction, and process the data


In [10]:
### row (metabolite) filtering ###
#delete metabolite or row if it has NA or empty value for hmdbID
mbxData<- mbxData[!(is.na(mbxData$HMDB...Representative.ID.) | mbxData$HMDB...Representative.ID.=="") , ]
#remove rows which has hmdb as "redundant ion"
mbxData<- mbxData[!(mbxData$HMDB...Representative.ID.=="redundant ion") , ]
#remove character (asterisk) in some hmdb column values
mbxData$HMDB...Representative.ID.<- stringr::str_replace(mbxData$HMDB...Representative.ID., '\\*', '')
#Update HMDB IDs to new data structure
mbxData$HMDB...Representative.ID.<- stringr::str_replace(mbxData$HMDB...Representative.ID., 'HMDB', 'HMDB00')
#back up original mbxdata
mbxData.b <- mbxData

### modify mbxData based on sample names given in metaData file (created with the criteria visit_num=4 )###
#filter out mbxData columns (samples) based metaDataMBX externalIDs
names.use <- names(mbxData)[ names(mbxData) %in% metaDataMBX$ExternalID]
#update mbx data with used names
mbxData <- mbxData [ ,names.use]
#order data based on col names
mbxData <- mbxData[ , order(names(mbxData))]

#order metadata based on externalID
metaDataMBX <- metaDataMBX[order(metaDataMBX$ExternalID),]

#add HMDBID and Compound Name column to the mbx data
mbxData <- cbind(mbxData.b$HMDB...Representative.ID., mbxData.b$Metabolite,mbxData)
colnames(mbxData)[1] <- "HMDB.ID"
colnames(mbxData)[2] <- "Compound.Name"

#add disease labels to the mbx data
diseaseLabels <- metaDataMBX$disease
##Add two empty strings to match with additional column data.
diseaseLabels <- append(diseaseLabels, "NA",after = 0)
diseaseLabels <- append(diseaseLabels, "NA",after = 0)
mbxData <- rbind(diseaseLabels, mbxData)
head(mbxData)
metaDataMBX$disease

Warning message in `[<-.factor`(`*tmp*`, ri, value = "NA"):
"invalid factor level, NA generated"Warning message in `[<-.factor`(`*tmp*`, ri, value = "NA"):
"invalid factor level, NA generated"

,HMDB.ID,Compound.Name,CSM5FZ3N,CSM5FZ3T,CSM5FZ48,CSM5FZ4A,CSM5FZ4O,CSM5MCTZ,CSM5MCU4,CSM5MCVH,...,MSM79H94,MSM79HD6,MSM9VZFJ,MSM9VZM4,MSM9VZMM,PSM6XBQM,PSM6XBRK,PSM6XBSE,PSM7J188,PSMA265X
1,NA,NA,CD,CD,UC,UC,UC,UC,CD,CD,...,nonIBD,UC,nonIBD,CD,nonIBD,CD,CD,UC,CD,UC
110,HMDB0004705,12.13-diHOME,942546,470132,1069292,1265301,524759,5745981,856432,8915552,...,3480357,3119960,946500,1237421,2481188,6295050,2384194,4071076,638602,1563896
2,HMDB0004704,9.10-diHOME,1418451,416924,1256069,1368667,541578,1260673,994782,9004716,...,1921336,1456450,825209,1594504,1503852,4113128,1436963,2909581,1093483,964505
3,HMDB0000535,caproate,2194,28,773,542,13205,424,2953,15,...,1815,11008,2396,2813,2145,1315,897,41,667,1003
4,HMDB0000666,heptanoate,NA,NA,NA,NA,5664,NA,NA,NA,...,NA,28,NA,NA,NA,65,NA,197,38,NA
5,HMDB0000764,hydrocinnamate,38180,11781,62098,3235743,1189143,3653962,15982,7274,...,200785,15343,1565763,208250,1026008,856105,264597,26166,483450,41530


[1] "CD"     "CD"     "UC"     "UC"     "UC"     "UC"     "CD"     "CD"    
 [9] "CD"     "UC"     "CD"     "UC"     "UC"     "CD"     "CD"     "nonIBD"
[17] "CD"     "CD"     "UC"     "CD"     "UC"     "CD"     "UC"     "UC"    
[25] "CD"     "CD"     "UC"     "CD"     "CD"     "CD"     "nonIBD" "CD"    
[33] "nonIBD" "CD"     "UC"     "nonIBD" "nonIBD" "nonIBD" "nonIBD" "CD"    
[41] "nonIBD" "UC"     "UC"     "UC"     "CD"     "CD"     "UC"     "nonIBD"
[49] "CD"     "CD"     "CD"     "CD"     "CD"     "nonIBD" "nonIBD" "nonIBD"
[57] "nonIBD" "nonIBD" "nonIBD" "nonIBD" "nonIBD" "UC"     "nonIBD" "CD"    
[65] "nonIBD" "CD"     "CD"     "UC"     "CD"     "UC"


## Fourth, we split up the data for UC and CD, include the control data nonIBD, and save this data to an output folder.


In [ ]:
#write only UC versus nonIBD comparison
mbxDataUC <- mbxData[ ,(mbxData[1, ] == "UC" | mbxData[1, ] == "nonIBD")]
#add hmdb id again
mbxDataUC <- cbind(mbxData[,1:2],mbxDataUC)
colnames(mbxDataUC)[1]="HMBDB.ID"
colnames(mbxDataUC)[2] <- "Compound.Name"
write.table(mbxDataUC, "output/mbxDataUC_nonIBD.csv", sep =",", row.names = FALSE)

#write only CD_healthy comparison
mbxDataCD <- mbxData[ ,(mbxData[1, ] == "CD" | mbxData[1, ] == "nonIBD")]
mbxDataCD <- cbind(mbxData[,1:2],mbxDataCD)
colnames(mbxDataCD)[1]="HMBDB.ID"
colnames(mbxDataCD)[2] <- "Compound.Name"
write.table(mbxDataCD, "output/mbxDataCD_nonIBD.csv", sep =",", row.names = FALSE)



### Last, we create a Jupyter notebook and markdown file from this script


In [ ]:
#Jupyter Notebook file
if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
library(devtools)
library(rmd2jupyter)
rmd2jupyter("dataPreprocessing.Rmd")
##Clean up R-studio environment
remove(diseaseLabels, fileUrl, names.use, mbxData, mbxData.b, mbxDataCD, mbxDataUC, metaDataMBX, metaData)
